## Fuzzy Matching School Names

Matches school name in educator data with their ids.

In [14]:
import os
import pandas as pd
from fuzzywuzzy import fuzz, process
from collections import defaultdict

In [15]:
input_data_dir = "data-to-match"

educators_inexperienced_path = os.path.join(input_data_dir, "Educators_Inexperienced_2018_JAN_24th_2019.csv")
educators_out_of_field_path = os.path.join(input_data_dir, "Educators_OUT_OF_FIELD_2018_JAN_24th_2019.csv")
school_id_data_path = os.path.join(input_data_dir, "ga_public_school_contact_list_Ethan.xlsx")

educators_inexperienced = pd.read_csv(educators_inexperienced_path)
educators_out_of_field = pd.read_csv(educators_out_of_field_path)
school_id_data = pd.read_excel(school_id_data_path, sheet_name="Sheet_1")


In [16]:
# Cleaning Public School Data.
school_id_data["SCHOOL_NAME"] = school_id_data["SCHOOL_NAME"].str.upper()
school_id_data["SCHOOL_NAME"] = school_id_data["SCHOOL_NAME"].str.strip()
school_id_data["SYSTEM_NAME"] = school_id_data["SYSTEM_NAME"].str.upper()
school_id_data["SYSTEM_NAME"] = school_id_data["SYSTEM_NAME"].str.strip()
# Must take care of duplicates separately. Some schools have the same name but are differet schools with different IDs.
school_id_data_no_duplicates = school_id_data.drop_duplicates(subset="SCHOOL_NAME", keep=False)

In [17]:
# Cleaning Educators Inexperienced.
educators_inexperienced["INSTN_NAME"] = educators_inexperienced["INSTN_NAME"].str.upper()
educators_inexperienced["INSTN_NAME"] = educators_inexperienced["INSTN_NAME"].str.strip()
educators_inexperienced["SCHOOL_DSTRCT_NM"] = educators_inexperienced["SCHOOL_DSTRCT_NM"].str.upper()
educators_inexperienced["SCHOOL_DSTRCT_NM"] = educators_inexperienced["SCHOOL_DSTRCT_NM"].str.strip()
educators_inexperienced = educators_inexperienced[~educators_inexperienced["INSTN_NAME"].str.endswith("ALL SCHOOLS")]

In [18]:
# Cleaning Educators Out of Field.
educators_out_of_field["INSTN_NAME"] = educators_out_of_field["INSTN_NAME"].str.upper()
educators_out_of_field["INSTN_NAME"] = educators_out_of_field["INSTN_NAME"].str.strip()
educators_out_of_field["SCHOOL_DSTRCT_NM"] = educators_out_of_field["SCHOOL_DSTRCT_NM"].str.upper()
educators_out_of_field["SCHOOL_DSTRCT_NM"] = educators_out_of_field["SCHOOL_DSTRCT_NM"].str.strip()
educators_out_of_field = educators_out_of_field[~educators_out_of_field["INSTN_NAME"].str.endswith("ALL SCHOOLS")]

### Matching for Inexperienced Educators

In [19]:
inexperienced_with_perfect_match = educators_inexperienced[educators_inexperienced["INSTN_NAME"].isin(school_id_data["SCHOOL_NAME"])]
inexperienced_no_perfect_match = educators_inexperienced[~educators_inexperienced["INSTN_NAME"].isin(school_id_data["SCHOOL_NAME"])]

print("Shape of inexperienced_with_perfect_match: {}".format(inexperienced_with_perfect_match.shape))
print("Shape of inexperienced_no_perfect_match: {}".format(inexperienced_no_perfect_match.shape))

Shape of inexperienced_with_perfect_match: (5587, 8)
Shape of inexperienced_no_perfect_match: (97, 8)


In [20]:
print("Number of schools in inexperienced data: {}".format(len(educators_inexperienced)))
print("Number of schools in inexperienced data with perfect match: {}".format(len(inexperienced_with_perfect_match)))
print("Number of schools in inexperienced data with no perfect match: {}".format(len(inexperienced_no_perfect_match)))

correct_size = inexperienced_with_perfect_match.shape[0] + inexperienced_no_perfect_match.shape[0] == educators_inexperienced.shape[0]
print("Sizes of perfect and non-perfect subsets is correct: {}".format(correct_size))

no_overlap = len(set(inexperienced_with_perfect_match["INSTN_NAME"]).intersection(set(inexperienced_no_perfect_match["INSTN_NAME"]))) == 0
print("No overlap between perfect and non-perfect matching subsets: {}".format(no_overlap))

Number of schools in inexperienced data: 5684
Number of schools in inexperienced data with perfect match: 5587
Number of schools in inexperienced data with no perfect match: 97
Sizes of perfect and non-perfect subsets is correct: True
No overlap between perfect and non-perfect matching subsets: True


In [21]:
merged_inexperienced = inexperienced_with_perfect_match.merge(
    school_id_data, # We can use the full dataset here because we are matching on both school name and county.
    left_on=["INSTN_NAME", "SCHOOL_DSTRCT_NM"],
    right_on=["SCHOOL_NAME", "SYSTEM_NAME"],
    how="left",
)

merged_inexperienced["MATCH_TYPE"] = "PERFECT"
merged_inexperienced["SCORE"] = 100

print("Shape of merged_inexperienced: {}".format(merged_inexperienced.shape))

Shape of merged_inexperienced: (5587, 31)


In [22]:
unmatched_by_county = merged_inexperienced[merged_inexperienced["Full School_ID"].isnull()]
print("Number of unmatched schools by county: {}".format(len(unmatched_by_county)))

dictionary = defaultdict(lambda : "")
for index, row in school_id_data_no_duplicates.iterrows():
    dictionary[row["SCHOOL_NAME"]] = row["Full School_ID"]

unmatched_by_county["Full School_ID"] = unmatched_by_county["INSTN_NAME"].map(dictionary)

unmatched_by_county["MATCH_TYPE"] = "NON_MATCHING_COUNTY"
unmatched_by_county["SCORE"] = 100

Number of unmatched schools by county: 272


C:\Users\ethan_haque\AppData\Local\Temp\ipykernel_27872\2087428240.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unmatched_by_county["Full School_ID"] = unmatched_by_county["INSTN_NAME"].map(dictionary)
C:\Users\ethan_haque\AppData\Local\Temp\ipykernel_27872\2087428240.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unmatched_by_county["MATCH_TYPE"] = "NON_MATCHING_COUNTY"


In [29]:
def fuzzy_match_school_name(name):
    """Matches school name to school id using fuzzy matching.
    
    Parameters
    ----------
    name : str
        School name to match.
        
    Returns
    -------
    name : str
        Tuple of matched name, match confidence score, school id, system name, and school name.
    """
    match = process.extractOne(name, school_id_data_no_duplicates["SCHOOL_NAME"])
    name = match[0]
    score = match[1]
    row = match[2]
    school_id = school_id_data_no_duplicates.loc[row]["Full School_ID"]
    system_name = school_id_data_no_duplicates.loc[row]["SYSTEM_NAME"]
    school_name = school_id_data_no_duplicates.loc[row]["SCHOOL_NAME"]
    return (name, score, school_id, system_name, school_name)

In [33]:
inexperienced_no_perfect_match["SCHOOL_ID"] = inexperienced_no_perfect_match["INSTN_NAME"].apply(fuzzy_match_school_name)
# Splitting the tuple into separate columns.
inexperienced_no_perfect_match[["SCHOOL_NAME", "SCORE", "Full School_ID", "SYSTEM_NAME", "SCHOOL_NAME"]] = pd.DataFrame(inexperienced_no_perfect_match["SCHOOL_ID"].tolist(), index=inexperienced_no_perfect_match.index)
inexperienced_no_perfect_match = inexperienced_no_perfect_match.drop("SCHOOL_ID", axis=1)

inexperienced_no_perfect_match["MATCH_TYPE"] = "FUZZY"

C:\Users\ethan_haque\AppData\Local\Temp\ipykernel_27872\2101684407.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inexperienced_no_perfect_match["SCHOOL_ID"] = inexperienced_no_perfect_match["INSTN_NAME"].apply(fuzzy_match_school_name)
C:\Users\ethan_haque\AppData\Local\Temp\ipykernel_27872\2101684407.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inexperienced_no_perfect_match[["SCHOOL_NAME", "SCORE", "Full School_ID", "SYSTEM_NAME", "SCHOOL_NAME"]] = pd.DataFrame(inexperienced_no_perfect_match["

In [34]:
inexperienced_no_perfect_match

,LONG_SCHOOL_YEAR,SCHOOL_DSTRCT_NM,INSTN_NAME,LABEL_LVL_3_DESC,LABEL_LVL_2_DESC,FTE,INEXPERIENCED_FTE,INEXPERIENCED_FTE_PCT,MATCH_TYPE,SCHOOL_NAME,SCORE,Full School_ID,SYSTEM_NAME
309,2017-18,BAKER COUNTY,BAKER COUNTY LEARNING CENTER,Leaders,High Poverty,0.3,0.3,100,FUZZY,CRISP COUNTY PRE-K,86,6405050,CRISP COUNTY
310,2017-18,BAKER COUNTY,BAKER COUNTY LEARNING CENTER,Teachers,High Poverty,1.0,1.0,100,FUZZY,CRISP COUNTY PRE-K,86,6405050,CRISP COUNTY
311,2017-18,BAKER COUNTY,BAKER COUNTY LEARNING CENTER,Teachers,Total,1.0,1.0,100,FUZZY,CRISP COUNTY PRE-K,86,6405050,CRISP COUNTY
745,2017-18,CALHOUN CITY,CALHOUN PRIMARY SCHOOL,Leaders,Not Applicable,2.0,0.0,0,FUZZY,BLECKLEY COUNTY ELEMENTARY SCHOOL,86,6120197,BLECKLEY COUNTY
746,2017-18,CALHOUN CITY,CALHOUN PRIMARY SCHOOL,Teachers,Total,63.2,23.6,37,FUZZY,BLECKLEY COUNTY ELEMENTARY SCHOOL,86,6120197,BLECKLEY COUNTY
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5919,2017-18,TIFT COUNTY,NORTHSIDE PRIMARY SCHOOL,Leaders,High Poverty,2.3,1.1,47,FUZZY,COOK HIGH SCHOOL,86,6370111,COOK COUNTY
5920,2017-18,TIFT COUNTY,NORTHSIDE PRIMARY SCHOOL,Teachers,High Poverty,31.7,10.0,32,FUZZY,COOK HIGH SCHOOL,86,6370111,COOK COUNTY
5921,2017-18,TIFT COUNTY,NORTHSIDE PRIMARY SCHOOL,Teachers,Total,31.7,10.0,32,FUZZY,COOK HIGH SCHOOL,86,6370111,COOK COUNTY
6280,2017-18,WHITE COUNTY,WHITE COUNTY 9TH GRADE ACADEMY,Leaders,Not Applicable,1.4,0.0,0,FUZZY,9TH GRADE ACADEMY,90,6680106,HABERSHAM COUNTY


### Matching for Out of Field Educators

### Saving Output

In [35]:
output_dir = "matched-data"

In [36]:
# rememeber to remove non-alphanumeric characters from school names.

inexperienced_matched = pd.concat([inexperienced_with_perfect_match, inexperienced_no_perfect_match])
inexperienced_matched[
    [
        "LONG_SCHOOL_YEAR",
        "SCHOOL_DSTRCT_NM",
        "SYSTEM_NAME",
        "INSTN_NAME",
        "SCHOOL_NAME",
        "LABEL_LVL_3_DESC",
        "LABEL_LVL_2_DESC",
        "FTE",
        "INEXPERIENCED_FTE",
        "INEXPERIENCED_FTE_PCT",
        "Full School_ID",
    ]
].to_csv("inexperienced-matched.csv", index=False)

# Cleaned by hand.

